In [1]:
import pickle
import pandas as pd
import os
import torch

import betfairlightweight
from betfairlightweight import filters
from datetime import datetime, timedelta
from datetime import timedelta
from dateutil import tz
import math
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.linear_model import LogisticRegression
import fasttrack as ft
import importlib
import featurecreations
from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder
#pd.set_option('display.max_rows', 100)
from ft_sec_key import SECKEY, TOPAZ
import xmltodict
import urllib
import requests_async as requests
# import httpx
import concurrent.futures as futures
import asyncio

In [2]:
from topaz import topaz
from ft_sec_key import SECKEY, TOPAZ
import pandas as pd
import importlib

In [3]:
topaz
importlib.reload(topaz)

<module 'topaz.topaz' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\python\\data_tools\\topaz\\topaz.py'>

In [4]:
topaz_api = topaz.TopazAPI(TOPAZ)

In [5]:
def chunks(xs, n):
    n = max(1, n)
    chunk_size = len(xs) // n
    return [xs[i : i + chunk_size] for i in range(0, len(xs), chunk_size)]

In [28]:
races = topaz_api.get_races(from_date="2023-01-01", to_date="2023-12-01", owning_authority_code='NZ')
print(len(races))

4695


In [29]:
races.shape

(4695, 43)

In [30]:
races.columns

Index(['raceId', 'meetingId', 'raceNumber', 'raceTypeCode', 'raceType', 'name',
       'startTime', 'raceStart', 'distance', 'isBoxDrawn', 'prizeMoney1',
       'prizeMoney2', 'prizeMoney3', 'prizeMoney4', 'prizeMoney5',
       'prizeMoney6', 'prizeMoney7', 'prizeMoney8', 'prizeMoneyTotal',
       'analystComment', 'suggestedBetTypeCode', 'suggestedBet',
       'isDailyDouble', 'isQuaddie', 'isEarlyQuaddie', 'isRunningDouble',
       'isTrifecta', 'isQuinella', 'isExacta', 'isPickFour', 'isBixSix',
       'tip1', 'tip2', 'tip3', 'tip4', 'winPool', 'placePool', 'trifectaPool',
       'owningAuthorityCode', 'isRaceResultEntered', 'photoFinishUrl',
       'resultYouTubeVideoId', 'tabTipRunners'],
      dtype='object')

In [31]:
race = topaz_api.get_race_runs(972428497)

In [32]:
race.columns

Index(['trackCode', 'track', 'distance', 'raceId', 'meetingDate',
       'raceTypeCode', 'raceType', 'runId', 'dogId', 'dogName', 'weightInKg',
       'incomingGrade', 'outgoingGrade', 'gradedTo', 'rating', 'raceNumber',
       'boxNumber', 'boxDrawnOrder', 'rugNumber', 'startPrice', 'place',
       'unplaced', 'unplacedCode', 'scratched', 'prizeMoney', 'resultTime',
       'resultMargin', 'resultMarginLengths', 'startPaceCode', 'jumpCode',
       'runLineCode', 'firstSecond', 'colourCode', 'sex', 'comment', 'ownerId',
       'trainerId', 'ownerName', 'ownerState', 'trainerName', 'trainerSuburb',
       'trainerState', 'trainerDistrict', 'trainerPostCode', 'isQuad',
       'isBestBet', 'damId', 'damName', 'sireId', 'sireName', 'dateWhelped',
       'totalFormCount', 'last5', 'isLateScratching', 'bestTime',
       'bestFinishTrackAndDistance', 'pir', 'careerPrizeMoney'],
      dtype='object')

In [39]:
race_results = topaz_api.get_meeting_first_split_results()

PermissionError: You are not authorized to access this endpoint.

In [ ]:
race_results

{'raceId': 972428497,
 'meetingId': 900012680,
 'raceNumber': 1,
 'raceTypeCode': 'M',
 'raceType': 'Maiden',
 'name': 'SPORTSBET EVERY DOG HAS ITS DAY (250+)',
 'startTime': '3:27PM',
 'raceStart': '2023-12-05T04:27:00.000Z',
 'distance': 400,
 'isBoxDrawn': True,
 'prizeMoney1': 1000,
 'prizeMoney2': 320,
 'prizeMoney3': 160,
 'prizeMoney4': 80,
 'prizeMoney5': 0,
 'prizeMoney6': 0,
 'prizeMoney7': 0,
 'prizeMoney8': 0,
 'prizeMoneyTotal': 1560,
 'analystComment': 'LITERALLY (6) was far from disgraced here two runs back and she will appreciate the empty draws inside. ELISSA BALE (7) has fair form and looks next best.  HAVE A LASH (3) is more than capable',
 'suggestedBetTypeCode': 'BoxQuinella',
 'suggestedBet': '3,6,7 ($10 for 333.33%)',
 'isDailyDouble': False,
 'isQuaddie': False,
 'isEarlyQuaddie': True,
 'isRunningDouble': True,
 'isTrifecta': True,
 'isQuinella': True,
 'isExacta': True,
 'isPickFour': True,
 'isBixSix': False,
 'tip1': 6,
 'tip2': 7,
 'tip3': 3,
 'tip4': 2,
 '

In [6]:
import os
import time
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd
from topaz import TopazAPI


# Generate a date range
def generate_date_range(start_date, end_date):
    start_date = start_date
    end_date = end_date

    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)

    return date_list

# Input the number of days of historical data required
# To input absolute dates rather than a timedelta use 'datetime(YYYY,M,D)'
start_date = (datetime.today() - timedelta(days=365))
end_date = (datetime.today() - timedelta(days=1))

# Generate the date range
date_range = generate_date_range(start_date, end_date)

# Iterate over 7-day blocks
for i in range(0, len(date_range), 6):
    start_block_date = date_range[i]
    print(start_block_date)
    end_block_date = date_range[min(i + 6, len(date_range) - 1)]  # Ensure the end date is within the range

    codes = ['NZ', 'NT', 'VIC', 'NSW', 'SA', 'WA', 'QLD', 'TAS']

    for code in codes:
        all_races = []
        print(code)
        retries = 10  # Number of retries
        while retries > 0:
            try:
                races = topaz_api.get_races(from_date=start_block_date, to_date=end_block_date, owning_authority_code=code)
                all_races.append(races)
                break  # Break out of the loop if successful
            except Exception as e:
                print(f"Error fetching races for {code}: {e}")
                retries -= 1
                if retries > 0:
                    print(f"Retrying in 30 seconds...")
                    time.sleep(30)
                else:
                    print("Max retries reached. Moving to the next block.")

        try:
            all_races_df = pd.concat(all_races, ignore_index=True)
        except ValueError:
            continue

        # Extract unique race IDs
        race_ids = list(all_races_df['raceId'].unique())

        for race_id in tqdm(race_ids, desc="Processing races", unit="race"):
            result_retries = 10

            while result_retries > 0:
                # Use tqdm to create a progress bar
                # Get race run data
                try:
                    race_run = topaz_api.get_race_runs(race_id=race_id)
                    file_path = code + '_DATA.csv'
                    file_exists = os.path.isfile(file_path)
                    header_param = not file_exists
                    race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
                    break
                except Exception as e:
                    result_retries -= 1
                    print(e)
                    if result_retries > 0:
                        time.sleep(15)
                    else:
                        time.sleep(120)

2023-01-30
NZ
Error fetching races for NZ: 404 Client Error: Not Found for url: https://topaz.grv.org.au/api/race?from=2023-01-30&to=2023-02-05&owningauthoritycode=NZ
Retrying in 30 seconds...


Processing races: 100%|██████████| 98/98 [02:37<00:00,  1.61s/race]


NT


Processing races: 100%|██████████| 8/8 [00:12<00:00,  1.51s/race]


VIC


...

VIC


Processing races: 100%|██████████| 295/295 [08:46<00:00,  1.78s/race]


NSW


Processing races:  36%|███▌      | 94/260 [02:43<04:48,  1.74s/race]


KeyboardInterrupt: 

In [ ]:
import os
import time
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd
from topaz import TopazAPI

# Generate a date range
def generate_date_range(start_date, end_date):
    start_date = start_date
    end_date = end_date

    date_list = []
    current_date = start_date
    while current_date < end_date:
        date_list.append(current_date)
        current_date += timedelta(days=7)
        current_date = min(current_date, end_date)

    return date_list

# Input the number of days of historical data required
# To input absolute dates rather than a timedelta use 'datetime(YYYY,M,D)'
start_date = (datetime.today() - timedelta(days=365))
end_date = (datetime.today() - timedelta(days=1))

# Generate the date range
date_range = generate_date_range(start_date, end_date)

date_range_chunks = chunks(date_range,10)
# date_range

# Iterate over 7-day blocks
# 

def process_races(date_list,max_date):
    for i,start_date in enumerate(date_list):
        start_block_date = start_date
        print(start_block_date)
        end_block_date = start_date + timedelta(days=6)

        codes = ['NZ', 'NT', 'VIC', 'NSW', 'SA', 'WA', 'QLD', 'TAS']
        end_block_date = end_block_date.strftime("%Y-%m-%d")
        start_block_date = start_block_date.strftime("%Y-%m-%d")

        for code in codes:
            all_races = []
            print(code)
            retries = 10  # Number of retries
            while retries > 0:
                try:
                    races = topaz_api.get_races(from_date=start_block_date, to_date=end_block_date, owning_authority_code=code)
                    all_races.append(races)
                    break  # Break out of the loop if successful
                except Exception as e:
                    print(f"Error fetching races for {code}: {e}")
                    if "404" in str(e):
                        break
                    retries -= 1
                    if retries > 0:
                        print(f"Retrying in 30 seconds...")
                        time.sleep(30)
                    else:
                        print("Max retries reached. Moving to the next block.")

            try:
                all_races_df = pd.concat(all_races, ignore_index=True)
            except ValueError:
                continue

            # Extract unique race IDs
            race_ids = list(all_races_df['raceId'].unique())

            for race_id in tqdm(race_ids, desc="Processing races", unit="race"):
                result_retries = 5

                while result_retries > 0:
                    # Use tqdm to create a progress bar
                    # Get race run data
                    try:
                        race_run = topaz_api.get_race_runs(race_id=race_id)
                        file_path = code + '_DATA.csv'
                        file_exists = os.path.isfile(file_path)
                        header_param = not file_exists
                        race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
                        break
                    except Exception:
                        result_retries -= 1
                        if result_retries > 0:
                            time.sleep(15)
                        else:
                            time.sleep(120)

_process_jobs = []
with futures.ThreadPoolExecutor(max_workers=5) as p:
    for i, m in enumerate(date_range_chunks):
        _process_jobs.append(
            p.submit(
                process_races,
                m, end_date
            )
        )
            # print(_process_jobs)
    for job in tqdm(futures.as_completed(_process_jobs), total=len(_process_jobs)):
        job.result()  # wait for result
        


2023-01-30 10:36:54.285188
NZ
2023-03-06 10:36:54.285188
NZ
2023-04-10 10:36:54.285188
NZ
2023-05-15 10:36:54.285188
NZ
2023-06-19 10:36:54.285188
NZ


  0%|          | 0/11 [00:00<?, ?it/s]

Error fetching races for NZ: 404 Client Error: Not Found for url: https://topaz.grv.org.au/api/race?from=2023-05-15&to=2023-05-21&owningauthoritycode=NZ
NT
Error fetching races for NZ: 404 Client Error: Not Found for url: https://topaz.grv.org.au/api/race?from=2023-01-30&to=2023-02-05&owningauthoritycode=NZ
NT


Error fetching races for NZ: 404 Client Error: Not Found for url: https://topaz.grv.org.au/api/race?from=2023-06-19&to=2023-06-25&owningauthoritycode=NZ
NT
Error fetching races for NZ: 404 Client Error: Not Found for url: https://topaz.grv.org.au/api/race?from=2023-04-10&to=2023-04-16&owningauthoritycode=NZ
NT
Error fetching races for NZ: 404 Client Error: Not Found for url: https://topaz.grv.org.au/api/race?from=2023-03-06&to=2023-03-12&owningauthoritycode=NZ
NT































































Processing races: 100%|██████████| 8/8 [00:13<00:00,  1.70s/race]


...

VIC





























Processing races: 100%|██████████| 16/16 [00:28<00:00,  1.75s/race]


VIC









Processing races: 100%|██████████| 16/16 [00:26<00:00,  1.65s/race]



VIC


In [ ]:
date_range_chunks

[['2021-01-30',
  '2021-02-06',
  '2021-02-13',
  '2021-02-20',
  '2021-02-27',
  '2021-03-06',
  '2021-03-13',
  '2021-03-20',
  '2021-03-27',
  '2021-04-03',
  '2021-04-10',
  '2021-04-17',
  '2021-04-24',
  '2021-05-01',
  '2021-05-08'],
 ['2021-05-15',
  '2021-05-22',
  '2021-05-29',
  '2021-06-05',
  '2021-06-12',
  '2021-06-19',
  '2021-06-26',
  '2021-07-03',
  '2021-07-10',
  '2021-07-17',
  '2021-07-24',
  '2021-07-31',
  '2021-08-07',
  '2021-08-14',
  '2021-08-21'],
 ['2021-08-28',
  '2021-09-04',
  '2021-09-11',
  '2021-09-18',
  '2021-09-25',
  '2021-10-02',
  '2021-10-09',
  '2021-10-16',
  '2021-10-23',
  '2021-10-30',
  '2021-11-06',
  '2021-11-13',
  '2021-11-20',
  '2021-11-27',
  '2021-12-04'],
 ['2021-12-11',
  '2021-12-18',
  '2021-12-25',
  '2022-01-01',
  '2022-01-08',
  '2022-01-15',
  '2022-01-22',
  '2022-01-29',
  '2022-02-05',
  '2022-02-12',
  '2022-02-19',
  '2022-02-26',
  '2022-03-05',
  '2022-03-12',
  '2022-03-19'],
 ['2022-03-26',
  '2022-04-02',
  '2

In [ ]:
import asyncio

def get_race_runs(race_id):
    return  topaz_api.get_race_runs(race_id=race_id)

async def main():
    races = topaz_api.get_races(from_date='2023-12-01', to_date='2023-12-06')
    race_ids = list(races['raceId'].unique())
    
    semaphore = asyncio.Semaphore(3)  # Limit concurrent requests to 3
    
    async def process_race(race_id):
        async with semaphore:
            race_runs = await get_race_runs(race_id)  # Add 'await' keyword here
            return race_runs
    
    race_runs = await asyncio.gather(*[process_race(race_id) for race_id in race_ids])
    return race_runs

async def get_race_runs(race_id):
    return await topaz_api.get_race_runs(race_id=race_id)

await main()


In [ ]:
races.owningAuthorityCode.unique()


array(['VIC'], dtype=object)

In [ ]:
import pandas as pd

authority_codes = [ 'NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA']
all_races = []

for code in authority_codes:
    try:
        races = topaz_api.get_races(from_date='2023-01-14', to_date='2023-01-30', owning_authority_code=code)#start on 18/1/24 next(last run on 23rd)
        all_races.append(races)
    except Exception as e:
        print(f"Error fetching races for {code}: {e}")

# Concatenate all fetched races into a single DataFrame
all_races_df = pd.concat(all_races, ignore_index=True)

# Extract unique race IDs
race_ids = list(all_races_df['raceId'].unique())

Error fetching races for TAS: 404 Client Error: Not Found for url: https://topaz.grv.org.au/api/race?from=2023-01-14&to=2023-01-30&owningauthoritycode=TAS


C:\Users\Nick\AppData\Local\Temp\ipykernel_25432\821475247.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_races_df = pd.concat(all_races, ignore_index=True)


In [ ]:
all_races_df.owningAuthorityCode.value_counts()

owningAuthorityCode
VIC    711
NSW    636
QLD    363
SA     236
WA     199
NT      35
Name: count, dtype: int64

In [ ]:
races = topaz_api.get_races(from_date='2023-12-01', to_date='2023-12-06')

In [ ]:
def xmldict(url):
    # Reads an xml file into as a python dict
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = urllib.request.Request(url, headers = hdr)
    file = urllib.request.urlopen(req)
    
    data = file.read()
    file.close()

    data = xmltodict.parse(data)
    return data

In [ ]:
async def async_xmldict(url):
    # Reads an xml file into as a python dict
    print('started')
    async with httpx.AsyncClient() as client:
        req = await client.get(url)
        data = req.text
        data = xmltodict.parse(data)
    print('finished')
    return data

In [ ]:
req = httpx.get('https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/24_04_2023/308/RaceResults/xml')

In [ ]:
data = async_xmldict('https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/24_04_2023/308/RaceResults/xml')
data

<coroutine object async_xmldict at 0x000001C7141761C0>

In [ ]:
urls = ['https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/24_04_2023/308/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/T/24_04_2023/313/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/27_04_2023/308/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/24_04_2023/308/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/T/24_04_2023/313/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/27_04_2023/308/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/24_04_2023/308/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/T/24_04_2023/313/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/27_04_2023/308/RaceResults/xml']

In [ ]:
todo = asyncio.Queue()


datas = []

for url in urls:
    await todo.put(url)

async def put_todo(url: str):
    await todo.put(url)

async def worker():
    while True:
        try:
            await process_one()
        except asyncio.CancelledError:
            return

async def process_one():
    url = await todo.get()
    try:
        data = await async_xmldict(url)
    except Exception as exc:
        # retry handling here...
        pass
    finally:
        todo.task_done()
        datas.append(data)


workers = [
    asyncio.create_task(worker())
    for _ in range(2)
]

await todo.join()

started
started
finished
started
finished
started
finished
started
finished
started
finished
started
finished
started
finished
started
finished
finished


In [ ]:
urls = ['https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/24_04_2023/308/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/T/24_04_2023/313/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/27_04_2023/308/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/24_04_2023/308/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/T/24_04_2023/313/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/27_04_2023/308/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/24_04_2023/308/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/T/24_04_2023/313/RaceResults/xml',
        'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/27_04_2023/308/RaceResults/xml']

In [ ]:
import fasttrack_async 

In [ ]:
importlib.reload(fasttrack_async)
fasttrack = fasttrack_async.Fasttrack(SECKEY)

Valid Security Key


In [ ]:
today = (datetime.today()-timedelta(days=1)).strftime('%Y-%m-%d')
lastweek = (datetime.today()-timedelta(days=2)).strftime('%Y-%m-%d')
print(lastweek,today)

2023-05-02 2023-05-03


In [ ]:
data = await fasttrack.getMeetingDetail(lastweek,today)

Getting meets for each date ..
None ['https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/02_05_2023', 'https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/03_05_2023']
here ??
here
<Queue maxsize=0 _queue=['https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/02_05_2023', 'https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/03_05_2023'] tasks=2>
here2
started
started
finished
finished
number of meetings = 2


100%|██████████| 2/2 [00:00<?, ?it/s]


In [ ]:
data

,track,timeslot,date
0,305,Twilight,02-May-2023
1,307,Twilight,02-May-2023
2,312,Night,02-May-2023
3,413,Night,02-May-2023
4,600,Morning,02-May-2023
5,600,Night,02-May-2023
6,511,Day,02-May-2023
7,203,Night,02-May-2023
8,205,Night,02-May-2023
9,704,Day,02-May-2023


In [ ]:
data = await fasttrack.getRaceResults(lastweek,today)

Getting meets for each date ..
None ['https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/02_05_2023', 'https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/03_05_2023']
here ??
here
<Queue maxsize=0 _queue=['https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/02_05_2023', 'https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/03_05_2023'] tasks=2>
here2
started
started
finished
finished
number of meetings = 2


100%|██████████| 2/2 [00:00<?, ?it/s]

[('305', 'Twilight', '02-May-2023'), ('307', 'Twilight', '02-May-2023'), ('312', 'Night', '02-May-2023'), ('413', 'Night', '02-May-2023'), ('600', 'Morning', '02-May-2023'), ('600', 'Night', '02-May-2023'), ('511', 'Day', '02-May-2023'), ('203', 'Night', '02-May-2023'), ('205', 'Night', '02-May-2023'), ('704', 'Day', '02-May-2023'), ('900', 'Day', '02-May-2023'), ('202', 'Twilight', '02-May-2023'), ('407', 'Day', '02-May-2023'), ('315', 'Day', '03-May-2023'), ('310', 'Night', '03-May-2023'), ('313', 'Night', '03-May-2023'), ('303', 'Twilight', '03-May-2023'), ('911', 'Day', '03-May-2023'), ('800', 'Night', '03-May-2023'), ('412', 'Night', '03-May-2023'), ('600', 'Morning', '03-May-2023'), ('502', 'Day', '03-May-2023'), ('200', 'Night', '03-May-2023'), ('214', 'Twilight', '03-May-2023'), ('400', 'Twilight', '03-May-2023'), ('404', 'Day', '03-May-2023'), ('600', 'Night', '03-May-2023'), ('218', 'Day', '03-May-2023'), ('244', 'Day', '03-May-2023')] ['https://fasttrack.grv.org.au/DataExpor

[('305', 'Twilight', '02-May-2023'), ('307', 'Twilight', '02-May-2023'), ('312', 'Night', '02-May-2023'), ('413', 'Night', '02-May-2023'), ('600', 'Morning', '02-May-2023'), ('600', 'Night', '02-May-2023'), ('511', 'Day', '02-May-2023'), ('203', 'Night', '02-May-2023'), ('205', 'Night', '02-May-2023'), ('704', 'Day', '02-May-2023'), ('900', 'Day', '02-May-2023'), ('202', 'Twilight', '02-May-2023'), ('407', 'Day', '02-May-2023'), ('315', 'Day', '03-May-2023'), ('310', 'Night', '03-May-2023'), ('313', 'Night', '03-May-2023'), ('303', 'Twilight', '03-May-2023'), ('911', 'Day', '03-May-2023'), ('800', 'Night', '03-May-2023'), ('412', 'Night', '03-May-2023'), ('600', 'Morning', '03-May-2023'), ('502', 'Day', '03-May-2023'), ('200', 'Night', '03-May-2023'), ('214', 'Twilight', '03-May-2023'), ('400', 'Twilight', '03-May-2023'), ('404', 'Day', '03-May-2023'), ('600', 'Night', '03-May-2023'), ('218', 'Day', '03-May-2023'), ('244', 'Day', '03-May-2023')] ['https://fasttrack.grv.org.au/DataExpor

In [ ]:
data = await fasttrack.getBasicFormat(lastweek,today)

Getting meets for each date ..
None ['https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/02_05_2023', 'https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/03_05_2023']
here ??
here
<Queue maxsize=0 _queue=['https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/02_05_2023', 'https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/03_05_2023'] tasks=2>
here2
started
started
finished
finished
number of meetings = 2


100%|██████████| 2/2 [00:00<?, ?it/s]

[('315', 'Day', '03-May-2023'), ('310', 'Night', '03-May-2023'), ('313', 'Night', '03-May-2023'), ('303', 'Twilight', '03-May-2023'), ('911', 'Day', '03-May-2023'), ('800', 'Night', '03-May-2023'), ('412', 'Night', '03-May-2023'), ('600', 'Morning', '03-May-2023'), ('502', 'Day', '03-May-2023'), ('200', 'Night', '03-May-2023'), ('214', 'Twilight', '03-May-2023'), ('400', 'Twilight', '03-May-2023'), ('404', 'Day', '03-May-2023'), ('600', 'Night', '03-May-2023'), ('218', 'Day', '03-May-2023'), ('244', 'Day', '03-May-2023'), ('305', 'Twilight', '02-May-2023'), ('307', 'Twilight', '02-May-2023'), ('312', 'Night', '02-May-2023'), ('413', 'Night', '02-May-2023'), ('600', 'Morning', '02-May-2023'), ('600', 'Night', '02-May-2023'), ('511', 'Day', '02-May-2023'), ('203', 'Night', '02-May-2023'), ('205', 'Night', '02-May-2023'), ('704', 'Day', '02-May-2023'), ('900', 'Day', '02-May-2023'), ('202', 'Twilight', '02-May-2023'), ('407', 'Day', '02-May-2023')] ['https://fasttrack.grv.org.au/DataExpor

[('315', 'Day', '03-May-2023'), ('310', 'Night', '03-May-2023'), ('313', 'Night', '03-May-2023'), ('303', 'Twilight', '03-May-2023'), ('911', 'Day', '03-May-2023'), ('800', 'Night', '03-May-2023'), ('412', 'Night', '03-May-2023'), ('600', 'Morning', '03-May-2023'), ('502', 'Day', '03-May-2023'), ('200', 'Night', '03-May-2023'), ('214', 'Twilight', '03-May-2023'), ('400', 'Twilight', '03-May-2023'), ('404', 'Day', '03-May-2023'), ('600', 'Night', '03-May-2023'), ('218', 'Day', '03-May-2023'), ('244', 'Day', '03-May-2023'), ('305', 'Twilight', '02-May-2023'), ('307', 'Twilight', '02-May-2023'), ('312', 'Night', '02-May-2023'), ('413', 'Night', '02-May-2023'), ('600', 'Morning', '02-May-2023'), ('600', 'Night', '02-May-2023'), ('511', 'Day', '02-May-2023'), ('203', 'Night', '02-May-2023'), ('205', 'Night', '02-May-2023'), ('704', 'Day', '02-May-2023'), ('900', 'Day', '02-May-2023'), ('202', 'Twilight', '02-May-2023'), ('407', 'Day', '02-May-2023')] ['https://fasttrack.grv.org.au/DataExpor

In [ ]:
data

(           @id RaceNum                   RaceName RaceTime    RaceTimeDateUTC  \
 0    896833091       1              SPORTSBET HT1  12:25PM  03 May 23 02:25AM   
 1    896833092       2              SPORTSBET HT2  12:42PM  03 May 23 02:42AM   
 2    896833089       3               TOPCAT VIDEO  01:07PM  03 May 23 03:07AM   
 3    896833088       4          themeadows.org.au  01:27PM  03 May 23 03:27AM   
 4    896833086       5      SKY RACING (1-3 WINS)  01:49PM  03 May 23 03:49AM   
 ..         ...     ...                        ...      ...                ...   
 306  894776678       8  ROWAN ANDERSON JOURNALIST  05:45PM  02 May 23 07:45AM   
 307  894776680       9  WWW.IPSWICHGREYHOUNDS.COM  06:02PM  02 May 23 08:02AM   
 308  894776682      10     IPSWICH LOCAL MAGAZINE  06:20PM  02 May 23 08:20AM   
 309  894776684      11                        TAB  06:35PM  02 May 23 08:35AM   
 310  894776686      12  WWW.IPSWICHGREYHOUNDS.COM  06:55PM  02 May 23 08:55AM   
 
     Distance 

In [ ]:
'https://fasttrack.grv.org.au/DataExport/50efd775-e988-4be3-924f-87631fabdc3f/d/03_05_2023/315/RaceResults/XML'
'https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/D/03_05_2023/315/RaceResults/xml'


In [ ]:
todo = asyncio.Queue()

datas = []

for url in urls:
    await todo.put(url)

async def put_todo(url: str):
    await todo.put(url)

async def worker():
    while True:
        try:
            await process_one()
        except asyncio.CancelledError:
            return

async def process_one():
    url = await todo.get()
    try:
        data = await async_xmldict(url)
    except Exception as exc:
        # retry handling here...
        pass
    finally:
        todo.task_done()
        datas.append(data)


workers = [
    asyncio.create_task(worker())
    for _ in range(2)
]

await todo.join()

started
started
finished
started
finished
started
finished
started
finished
started
finished
started
finished
started
finished
started
finished
finished


In [ ]:
len(datas)

9

In [ ]:
await todo.join()

for worker in workers:
    worker.cancel()


In [ ]:
for u in urls:
    await async_xmldict(u)

In [ ]:
tasks = [async_xmldict(item) for item in urls]
results = await asyncio.gather(*tasks, return_exceptions=True)

started
started
started
finished
finished
finished


In [ ]:
results

[{'Meet': {'@id': '882772030',
   'Track': 'Shepparton',
   'Date': '24 Apr 23',
   'Race': [{'@id': '893492919',
     'RaceNum': '1',
     'RaceName': 'SPORTSBET FINAL (VICGREYS)',
     'RaceTime': '10:46AM',
     'Distance': '450m',
     'RaceGrade': 'Maiden',
     'Dog': [{'@id': '689724699',
       'Place': '1',
       'DogName': 'GOT ATTITUDE',
       'Trainer': {'@id': '129459', '#text': 'J Thompson'},
       'Box': '5',
       'Rug': '5',
       'Weight': '32.5',
       'StartPrice': '$2.00F',
       'Handicap': None,
       'Margin1': '4.46',
       'Margin2': None,
       'PIR': 'Q/11',
       'Checks': '0',
       'Comments': None,
       'SplitMargin': '6.62',
       'RunTime': '25.36',
       'Prizemoney': None},
      {'@id': '723529676',
       'Place': '2',
       'DogName': 'KRAKEN FORCE',
       'Trainer': {'@id': '104873', '#text': 'K Eyles'},
       'Box': '3',
       'Rug': '3',
       'Weight': '32.1',
       'StartPrice': '$14.80',
       'Handicap': None,
       

In [ ]:
raceDet = xmldict('https://fasttrack.grv.org.au/dataexport/50efd775-e988-4be3-924f-87631fabdc3f/N/26_04_2023/302/RaceResults/xml')

In [ ]:
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")

with open(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\DATA\gru_inputs_new_simple_test.npy', 'rb') as f:
    data, stats_cols = pickle.load(f)

In [ ]:
data.columns

Index(['dogid', 'dog_name', 'raceid', 'race_grade', 'date', 'race_time',
       'race_num', 'trackOHE', 'track_name', 'state', 'track_code', 'dist',
       'bfSP', 'runtime', 'prev_race', 'prev_race_date', 'prev_race_track',
       'prev_race_state', 'place', 'margin', 'box', 'stats'],
      dtype='object')

In [ ]:
print(f"Latest date = {pd.to_datetime(data.date).max()}")

Latest date = 2023-04-20 00:00:00


In [ ]:
data['stats_cols'] = str([stats_cols])
data['stats_cols'] = data['stats_cols'].astype('category')

AttributeError: 'str' object has no attribute 'astype'

In [ ]:
data['stats_cols'].memory_usage()

9711776

In [ ]:
with open(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\testing new outs simple 2023-04-18.npy', 'rb') as f:
    data, stats_cols = pickle.load(f)

old = data

In [ ]:
race_id = '889068366'

pred_race = old[old['raceid']==race_id]
train_race = data[data['raceid']==race_id]

In [ ]:
compare = pd.DataFrame(data=[train_race.sort_values('box').stats.tolist(),pred_race.sort_values('box').stats.tolist()])

In [ ]:
pred_l  = pred_race.sort_values('box').stats.tolist()
train_l = train_race.sort_values('box').stats.tolist()

In [ ]:
pred_race.sort_values('box').stats.tolist()[0]==train_race.sort_values('box').stats.tolist()[0]

False

In [ ]:
pred_race.sort_values('box').stats.tolist()[0]

[1.0,
 1.0,
 0.0,
 0.0,
 35.9,
 1.0,
 16.480446927374302,
 -1.0,
 -1.0,
 8.1,
 0.0,
 1.0,
 64.0,
 3.0,
 35.9,
 17.9,
 -1.0]

In [ ]:
train_race.sort_values('box').stats.tolist()[0]

[1.0,
 1.0,
 0.0,
 0.0,
 35.7,
 1.0,
 16.480446927374302,
 -1.0,
 -1.0,
 8.1,
 0.0,
 1.0,
 64.0,
 3.0,
 35.9,
 17.9,
 -1.0]

In [ ]:
# pred_race.sort_values('box').stats.tolist()

In [ ]:
# train_race.sort_values('box').stats.tolist()

In [ ]:
spark = data[data['track_name'] == 'Sandown Park']

In [ ]:
sdog = data[data['dog_name'] == 'DARNUM DIVA']

In [ ]:
april_data = data[data['date']>datetime.strptime("2023-04-01", "%Y-%m-%d").date()]

In [ ]:
prev_full = pd.read_feather(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\results-df-merged-prices_cut.fth")

In [ ]:
prev_full.columns

Index(['DogId', 'DogName', 'RaceId', 'RaceGrade', 'dateF', 'dateF1', 'date',
       'RaceTime', 'RaceNum', 'Track', 'State', 'track_code', 'dist_x', 'BSP',
       'Place', 'place', 'margin', 'box', 'speed', 'split_speed',
       'split_margins', 'first_out', 'pos_change', 'weight', 'RunTime',
       'SplitTimes', 'StartPrice', 'DogGrade', 'race_time', 'tracksOnehot'],
      dtype='object')

In [ ]:
benders = prev_full[['Track','dist_x', 'SplitTimes','dateF']][prev_full['Track']=='Bendigo']

In [ ]:
times = prev_full[['date', 'SplitTimes', 'split_speed', 'split_margins']].groupby('date', sort=False).mean()

In [ ]:
prev_full_cut = prev_full[['DogId',
               'DogName',
               'RaceId',
               'RaceGrade',
               'dateF',
               'dateF1',
               'date',
               'RaceTime',
               'RaceNum',
               'Track',
               'State',
               'track_code',
               'dist_x',
               'BSP',
               'Place',
               'place',
               'margin',
               'box',
               'speed',
               'split_speed',
               'split_margins',
               'first_out',
               'pos_change',
               'weight',
               'RunTime',
               'SplitTimes',
               'StartPrice',
               'DogGrade']]

In [ ]:
prev_full_cut['RaceId'] = prev_full_cut['RaceId'].astype('category')

C:\Users\Nick\AppData\Local\Temp\ipykernel_11552\506664480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_full_cut['RaceId'] = prev_full_cut['RaceId'].astype('category')


In [ ]:
prev_full_cut['RaceId']

0        501007024
1        501007024
2        501007024
3        501007024
4        501007024
           ...    
22484    889068366
22487    889068366
22490    889068366
22493    889068462
22496    889068463
Name: RaceId, Length: 1211515, dtype: category
Categories (169816, object): ['498695480', '500539128', '500539129', '500539130', ..., '891347644', '891347645', '891347646', '891347647']

In [ ]:
prev_full_cut.astype('categorical')

TypeError: data type 'categorical' not understood

In [ ]:
ohe = pickle.load(open('encoder_new', 'rb'))

In [ ]:
prev_full_cut['Track'].drop_duplicates().reset_index()

,index,Track
0,0,Healesville
1,35114,Sale
2,55401,Sandown Park
3,66594,Dubbo
4,79085,Murray Bridge
...,...,...
64,1122892,Lithgow
65,1124508,Potts Park
66,1125300,Richmond Straight
67,1132787,Traralgon


In [ ]:
pd.to_numeric(prev_full_cut['RunTime'])

0        19.24
1        19.66
2        19.73
3        19.84
4        20.04
         ...  
22484    17.99
22487    18.08
22490    18.15
22493    19.87
22496    31.92
Name: RunTime, Length: 1211515, dtype: float64

In [ ]:
prev_full_cut['StartPrice'] = pd.to_numeric(prev_full_cut['StartPrice'].apply(lambda x: x.replace('$','').replace('F','') if isinstance(x,str) else x), errors='coerce').fillna(0)

C:\Users\Nick\AppData\Local\Temp\ipykernel_11552\2622415636.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_full_cut['StartPrice'] = pd.to_numeric(prev_full_cut['StartPrice'].apply(lambda x: x.replace('$','').replace('F','') if isinstance(x,str) else x), errors='coerce').fillna(0)


In [ ]:
pd.to_numeric(prev_full_cut['StartPrice'])

ValueError: Unable to parse string "$3.60" at position 1163870

In [ ]:
prev_full_cut.dtypes

DogId                    object
DogName                  object
RaceId                 category
RaceGrade                object
dateF                    object
dateF1           datetime64[ns]
date                     object
RaceTime                 object
RaceNum                  object
Track                    object
State                    object
track_code               object
dist_x                  float64
BSP                     float64
Place                   float64
place                   float64
margin                  float64
box                       int64
speed                   float64
split_speed             float64
split_margins           float64
first_out               float64
pos_change              float64
weight                  float64
RunTime                  object
SplitTimes              float64
StartPrice               object
DogGrade                float64
dtype: object

In [ ]:
prev_full_cut.memory_usage().sum()

281071480

In [ ]:
prev_full_cut = prev_full_cut.infer_objects()

In [ ]:
prev_full_cut.dtypes

DogId                    object
DogName                  object
RaceId                   object
RaceGrade                object
dateF                    object
dateF1           datetime64[ns]
date                     object
RaceTime                 object
RaceNum                  object
Track                    object
State                    object
track_code               object
dist_x                  float64
BSP                     float64
Place                   float64
place                   float64
margin                  float64
box                       int64
speed                   float64
split_speed             float64
split_margins           float64
first_out               float64
pos_change              float64
weight                  float64
RunTime                  object
SplitTimes              float64
StartPrice               object
DogGrade                float64
dtype: object

In [ ]:
print(prev_full.columns,prev_full_cut.columns)

Index(['@id_x', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice',
       'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments',
       'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId',
       'TrainerName', 'DogId', '@id_y', 'RaceNum', 'RaceName', 'RaceTime',
       'Distance', 'RaceGrade', 'Track_ft', 'State', 'track_code', 'date',
       'DogGrade', 'check', 'SplitTimes', 'minMargin', 'tracksOnehot',
       'dist_x', 'run_time', 'split_margins', 'place', 'track_id',
       'Unnamed: 0', 'dist_y', 'mean_time', 'mean_split', 'split_dist_estim',
       'split_dist', 'dateF1', 'speed', 'split_speed', 'box', 'margin',
       'weigth', 'dateF', 'dog_name', 'EVENT_ID', 'EVENT_DT', 'SELECTION_ID',
       'BSP', 'dog', 'MENU_HINT', 'weight', 'dist_round', 'Track',
       'StartPrice_num', 'PIR_adj', 'start_pos', 'last_pos', 'first_out',
       'pos_change', 'race_time'],
      dtype='object') Index(['DogId', 'DogName', 'RaceId', 'RaceGrade', 'dateF', 'dateF1', 'da

In [ ]:
prev_full.memory_usage().sum()

688140520

In [ ]:
prev_full_cut.memory_usage().sum()/prev_full.memory_usage().sum()

0.4084507042253521

In [ ]:
prev_full_cut.columns

Index(['DogId', 'DogName', 'RaceId', 'RaceGrade', 'dateF', 'dateF1', 'date',
       'RaceTime', 'RaceNum', 'Track', 'State', 'track_code', 'dist_x', 'BSP',
       'Place', 'place', 'margin', 'box', 'speed', 'split_speed',
       'split_margins', 'first_out', 'pos_change', 'weight', 'RunTime',
       'SplitTimes', 'StartPrice', 'DogGrade'],
      dtype='object')

In [ ]:
prev_full_cut_dtype = prev_full_cut

DogId                    object
DogName                  object
RaceId                   object
RaceGrade                object
dateF                    object
dateF1           datetime64[ns]
date                     object
RaceTime                 object
RaceNum                  object
Track                    object
State                    object
track_code               object
dist_x                  float64
BSP                     float64
Place                   float64
place                   float64
margin                  float64
box                       int64
speed                   float64
split_speed             float64
split_margins           float64
first_out               float64
pos_change              float64
weight                  float64
RunTime                  object
SplitTimes              float64
StartPrice               object
DogGrade                float64
dtype: object

In [ ]:
prev_full_cut_dtype = prev_full_cut

In [ ]:
prev_full_cut.to_pickle(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\results-df-merged-prices_cut.npy")
prev_full_cut.reset_index(drop=True).to_csv(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\results-df-merged-prices_cut.csv")

In [ ]:
prev_full_cut.reset_index(drop=True).to_feather(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\results-df-merged-prices_cut.fth")

In [ ]:
prev_full_cut_2 = pd.read_feather(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\results-df-merged-prices_cut.fth")

In [ ]:
prev_full_cut_2

,DogId,DogName,RaceId,RaceGrade,dateF,dateF1,date,RaceTime,RaceNum,Track,...,split_margins,first_out,pos_change,weight,RunTime,SplitTimes,StartPrice,DogGrade,race_time,tracksOnehot
0,314367490,SUPER BIG,501007024,Mixed 6/7,2019-12-01,2019-12-01,01 Dec 19,11:00AM,1,Healesville,...,0.00,0.0,0.0,32.0,19.24,0.00,4.7,6.0,2023-04-21 11:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,296096022,RIO DIXIE,501007024,Mixed 6/7,2019-12-01,2019-12-01,01 Dec 19,11:00AM,1,Healesville,...,0.12,0.0,0.0,32.2,19.66,0.12,2.8,6.0,2023-04-21 11:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,304203706,COCO JUMBO,501007024,Mixed 6/7,2019-12-01,2019-12-01,01 Dec 19,11:00AM,1,Healesville,...,0.17,0.0,0.0,29.9,19.73,0.17,3.7,7.0,2023-04-21 11:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,291188715,BEARING GOLD,501007024,Mixed 6/7,2019-12-01,2019-12-01,01 Dec 19,11:00AM,1,Healesville,...,0.23,0.0,0.0,29.2,19.84,0.23,32.0,7.0,2023-04-21 11:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,317789815,ROLLIN' IN,501007024,Mixed 6/7,2019-12-01,2019-12-01,01 Dec 19,11:00AM,1,Healesville,...,0.23,0.0,0.0,28.2,20.04,0.23,15.8,7.0,2023-04-21 11:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213244,566432214,APOLLO FUN,889068428,Mixed 4/5,2023-04-20,NaT,20 Apr 23,05:01PM,12,Northam,...,NaN,0.0,1.0,31.4,17.63,NaN,6.3,4.0,2023-04-21 17:01:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1213245,663002949,FLYNN MCCOOL,889068428,Mixed 4/5,2023-04-20,NaT,20 Apr 23,05:01PM,12,Northam,...,NaN,0.0,-1.0,30.2,17.73,NaN,6.9,4.0,2023-04-21 17:01:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1213246,495345715,NINO TONIC,889068428,Mixed 4/5,2023-04-20,NaT,20 Apr 23,05:01PM,12,Northam,...,NaN,0.0,0.0,35.7,17.81,NaN,19.9,5.0,2023-04-21 17:01:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1213247,537858594,EMMA MONELLI,889068428,Mixed 4/5,2023-04-20,NaT,20 Apr 23,05:01PM,12,Northam,...,NaN,0.0,-2.0,26.0,17.83,NaN,6.5,5.0,2023-04-21 17:01:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
prev_full_cut_2 = pd.read_csv(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\results-df-merged-prices_cut.csv",index_col=0)

NameError: name 'pd' is not defined

In [ ]:
prev_full_cut_2.memory_usage().sum

NameError: name 'prev_full_cut_2' is not defined

In [ ]:
req_columns = ['dist_x','box','speed','split_speed','split_margins','margin','first_out','pos_change','weight','RunTime','SplitTimes','StartPrice_num']

In [ ]:
other_columns = ['DogId', 'DogName', 'RaceId', 'RaceGrade', 'dateF','race_time','RaceNum','tracksOnehot','Track','State','track_code','dist_x', 'BSP','RunTime','Place','margin','box',]

In [ ]:
total = ['DogId', 'DogName', 'RaceId', 'RaceGrade', 'dateF','race_time','RaceNum','tracksOnehot','Track','State','track_code','dist_x', 'BSP','RunTime','Place','margin','box','dist_x','box','speed','split_speed','split_margins','margin','first_out','pos_change','weight','RunTime','SplitTimes','StartPrice_num']

In [ ]:
len(other_columns+req_columns)

29

In [ ]:
mem = prev_full.memory_usage()

In [ ]:
sdp = prev_full[prev_full['Track'].isin(['Sandown Park','Healesville'])]
sdp

,@id_x,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,...,weight,dist_round,Track,StartPrice_num,PIR_adj,start_pos,last_pos,first_out,pos_change,race_time
0,NaN,1.0,SUPER BIG,6,6,32.0,4.7,None,5.91,None,...,32.0,350.0,Healesville,4.7,1,1,1,0.0,0.0,2023-04-13 11:00:00
1,NaN,2.0,RIO DIXIE,1,1,32.2,2.8,None,5.91,5.91,...,32.2,350.0,Healesville,2.8,3,3,3,0.0,0.0,2023-04-13 11:00:00
2,NaN,3.0,COCO JUMBO,8,8,29.9,3.7,None,6.99,1.07,...,29.9,350.0,Healesville,3.7,6,6,6,0.0,0.0,2023-04-13 11:00:00
3,NaN,4.0,BEARING GOLD,4,4,29.2,32.0,None,8.46,1.47,...,29.2,350.0,Healesville,32.0,8,8,8,0.0,0.0,2023-04-13 11:00:00
4,NaN,5.0,ROLLIN' IN,3,3,28.2,15.8,None,11.31,2.86,...,28.2,350.0,Healesville,15.8,7,7,7,0.0,0.0,2023-04-13 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65534,NaN,3.0,ILIOPSOAS,7,7,29.7,$10.40,None,4.9,4.37,...,29.7,300.0,Healesville,10.4,5,5,5,0.0,0.0,2023-04-13 14:50:00
65539,NaN,4.0,OUR RUBY,1,1,27.8,$20.70,None,4.91,0.01,...,27.8,300.0,Healesville,20.7,4,4,4,0.0,0.0,2023-04-13 14:50:00
65544,NaN,5.0,IDAHO EVIE,6,6,26.3,$57.30,None,5.59,0.67,...,26.3,300.0,Healesville,57.3,2,2,2,0.0,0.0,2023-04-13 14:50:00
65549,NaN,6.0,BROADSIDE,8,8,35.2,$32.50,None,6.39,0.80,...,35.2,300.0,Healesville,32.5,7,7,7,0.0,0.0,2023-04-13 14:50:00


In [ ]:
bourne

,dogid,dog_name,raceid,race_grade,date,race_time,race_num,trackOHE,track_name,state,...,bfSP,runtime,prev_race,prev_race_date,prev_race_track,prev_race_state,place,margin,box,stats
286133,434800552,BOURNE BLUE,597578437,Maiden,2020-10-08,2023-04-11 12:07:00,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Mount Gambier,SA,...,3.834036,24.72,-1,-1,-1,-1,6.0,5.25,2,"[2.0, 1.0, 0.0, 0.0, 25.9, 8.0, -1.0, -1.0, -1..."
293421,434800552,BOURNE BLUE,599542826,Maiden,2020-10-15,2023-04-11 11:52:00,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Mount Gambier,SA,...,10.500000,17.07,597578437,2020-10-08 00:00:00,Mount Gambier,SA,4.0,4.75,1,"[1.0, 1.0, 0.0, 0.0, 25.8, 8.0, 400.0, 2.0, 16..."
307752,434800552,BOURNE BLUE,601881794,Maiden,2020-10-29,2023-04-11 11:57:00,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Mount Gambier,SA,...,65.000000,31.19,599542826,2020-10-15 00:00:00,Mount Gambier,SA,6.0,13.25,4,"[4.0, 0.0, 1.0, 0.0, 25.2, 8.0, 277.0, 1.0, 16..."
314895,434800552,BOURNE BLUE,606679026,Maiden,2020-11-05,2023-04-11 11:41:00,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Mount Gambier,SA,...,47.580399,23.82,601881794,2020-10-29 00:00:00,Mount Gambier,SA,3.0,2.25,2,"[2.0, 1.0, 0.0, 0.0, 25.4, 8.0, 512.0, 4.0, 16..."
321572,434800552,BOURNE BLUE,609365867,Maiden,2020-11-12,2023-04-11 12:30:00,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Mount Gambier,SA,...,30.000000,24.00,606679026,2020-11-05 00:00:00,Mount Gambier,SA,3.0,5.75,1,"[1.0, 1.0, 0.0, 0.0, 25.2, 8.0, 400.0, 2.0, 16..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175965,434800552,BOURNE BLUE,880900793,Grade 5,2023-03-13,2023-04-11 14:57:00,12,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Warrnambool,VIC,...,120.000000,23.25,878689878,2023-03-06 00:00:00,Warrnambool,VIC,5.0,5.71,3,"[3.0, 0.0, 1.0, 0.0, 25.8, 5.0, 390.0, 6.0, 17..."
1183625,434800552,BOURNE BLUE,882730967,Grade 5,2023-03-21,2023-04-11 18:51:00,12,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Warrnambool,VIC,...,NaN,22.95,880900793,2023-03-13 00:00:00,Warrnambool,VIC,6.0,2.69,3,"[3.0, 0.0, 1.0, 0.0, 25.7, 5.0, 390.0, 3.0, 16..."
1192629,434800552,BOURNE BLUE,884739611,Restricted Win,2023-03-30,2023-04-11 14:57:00,11,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Mount Gambier,SA,...,NaN,23.79,882730967,2023-03-21 00:00:00,Warrnambool,VIC,3.0,9.25,1,"[1.0, 1.0, 0.0, 0.0, 25.6, 5.0, 390.0, 3.0, 16..."
1196130,434800552,BOURNE BLUE,887081125,Grade 5,2023-04-03,2023-04-11 18:46:00,12,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Warrnambool,VIC,...,NaN,23.37,884739611,2023-03-30 00:00:00,Mount Gambier,SA,6.0,13.29,8,"[8.0, 0.0, 0.0, 1.0, 25.4, 5.0, 400.0, 1.0, 16..."


In [ ]:
ft_races = pd.read_pickle(r'./DATA/basic form/ft_races.npy')

In [ ]:
ft_races.columns

Index(['DogId', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice',
       'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments',
       'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId',
       'TrainerName', 'RaceNum', 'RaceName', 'RaceTime', 'Distance',
       'RaceGrade', 'Track_ft', 'Track', 'State', 'track_code', 'date',
       'DogGrade'],
      dtype='object')

In [ ]:
ft_races_sdp = ft_races[ft_races['Track'].isin(['Sandown Park','Healesville'])]

In [ ]:
ft_races_sdp

,DogId,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,...,RaceName,RaceTime,Distance,RaceGrade,Track_ft,Track,State,track_code,date,DogGrade
0,716368443,1,MANILA HECKLE,3,3,32.7,$7.10,None,1.69,None,...,SPORTSBET FINAL,06:57PM,515m,Maiden Final,Sandown Park,Sandown Park,VIC,301,06 Apr 23,M
1,716368443,1,MANILA HECKLE,3,3,32.7,$7.10,None,1.69,None,...,SPORTSBET FINAL,06:57PM,515m,Maiden Final,Sandown Park,Sandown Park,VIC,301,06 Apr 23,M
2,716368443,1,MANILA HECKLE,3,3,32.7,$7.10,None,1.69,None,...,SPORTSBET FINAL,06:57PM,515m,Maiden Final,Sandown Park,Sandown Park,VIC,301,06 Apr 23,M
3,716368443,1,MANILA HECKLE,3,3,32.7,$7.10,None,1.69,None,...,SPORTSBET FINAL,06:57PM,515m,Maiden Final,Sandown Park,Sandown Park,VIC,301,06 Apr 23,M
4,716368443,1,MANILA HECKLE,3,3,32.7,$7.10,None,1.69,None,...,SPORTSBET FINAL,06:57PM,515m,Maiden Final,Sandown Park,Sandown Park,VIC,301,06 Apr 23,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77240,675306991,7,VAILA BALE,2,2,26.0,$31.60,None,9.59,3.20,...,SPORTSBET GREEN TICKS,02:50PM,300m,Mixed 6/7,Healesville,Healesville,VIC,306,12 Apr 23,7
77241,675306991,7,VAILA BALE,2,2,26.0,$31.60,None,9.59,3.20,...,SPORTSBET GREEN TICKS,02:50PM,300m,Mixed 6/7,Healesville,Healesville,VIC,306,12 Apr 23,7
77242,675306991,7,VAILA BALE,2,2,26.0,$31.60,None,9.59,3.20,...,SPORTSBET GREEN TICKS,02:50PM,300m,Mixed 6/7,Healesville,Healesville,VIC,306,12 Apr 23,7
77243,675306991,7,VAILA BALE,2,2,26.0,$31.60,None,9.59,3.20,...,SPORTSBET GREEN TICKS,02:50PM,300m,Mixed 6/7,Healesville,Healesville,VIC,306,12 Apr 23,7


In [ ]:
ft_races.RaceId.value_counts()

881636270    120
881636276    120
880939395    120
877723427    120
877723434    120
            ... 
880939369     60
877658193     60
880939371     60
880939372     48
880939370     48
Name: RaceId, Length: 143, dtype: int64

In [ ]:
ft_races['StartPrice'].str.replace('\$','').replace('F','')

C:\Users\Nick\AppData\Local\Temp\ipykernel_22108\3503987892.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  ft_races['StartPrice'].str.replace('\$','')


0        2.90F
1        2.90F
2        2.90F
3        2.90F
4        2.90F
         ...  
13603     None
13604     None
13605     None
13606     None
13607     None
Name: StartPrice, Length: 13608, dtype: object

In [ ]:
bfdf = bfdf.sort_values('DATE')

In [ ]:
tailbf = bfdf.tail(1000)

In [ ]:
gru_in = pd.read_pickle(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\new_windows_gru_REAL_new.npy")

In [ ]:
hotrod = gru_in[gru_in['dogid']=='171126622']

In [ ]:
hotrod

,dogid,dog_name,raceid,race_grade,date,trackOHE,track_name,state,track_code,dist,bfSP,runtime,prev_race,prev_race_date,prev_race_track,prev_race_state,place,margin,stats
516,171126622,THRILLING KATIE,500543143,Class 4 Final,2019-12-01,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Auckland,NZ,903,318.0,NaN,19.27,-1,-1,-1,-1,6.0,10.92,"[6.0, 27.7, 4.0, -1.0, -1.0, -1.0, -1.0, -1.0,..."
14126,171126622,THRILLING KATIE,504087404,Class 4,2019-12-15,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Auckland,NZ,903,318.0,NaN,18.96,500543143,2019-12-01 00:00:00,Auckland,NZ,4.0,6.35,"[4.0, 27.4, 4.0, 16.50233523611832, 16.5023352..."
17384,171126622,THRILLING KATIE,504053460,Class 3,2019-12-19,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Auckland,NZ,903,318.0,NaN,19.51,504087404,2019-12-15 00:00:00,Auckland,NZ,8.0,9.87,"[7.0, 27.7, 3.0, 16.637243567426246, 16.772151..."
86186,171126622,THRILLING KATIE,530315519,Mixed Grade 2/3,2020-03-05,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Auckland,NZ,903,318.0,NaN,19.34,504053460,2019-12-19 00:00:00,Auckland,NZ,5.0,13.84,"[7.0, 26.9, 3.0, 16.52460693663031, 16.7721518..."
88150,171126622,THRILLING KATIE,530315592,Mixed 3/4,2020-03-08,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Auckland,NZ,903,318.0,NaN,18.82,530315519,2020-03-05 00:00:00,Auckland,NZ,3.0,1.64,"[8.0, 26.8, 3.0, 16.50410670195567, 16.7721518..."
94758,171126622,THRILLING KATIE,530388393,Mixed Grade 2/3,2020-03-15,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Auckland,NZ,903,318.0,NaN,19.65,530315592,2020-03-08 00:00:00,Auckland,NZ,8.0,19.61,"[5.0, 27.4, 3.0, 16.582668995995995, 16.896918..."


In [ ]:
tail = gru_in.tail(1000)

In [ ]:
gru_in.columns

Index(['dogid', 'dog_name', 'raceid', 'race_grade', 'date', 'trackOHE',
       'track_name', 'state', 'track_code', 'dist', 'bfSP', 'runtime',
       'prev_race', 'prev_race_date', 'prev_race_track', 'prev_race_state',
       'place', 'margin', 'stats'],
      dtype='object')

In [ ]:
r_stats = gru_in[['dogid','raceid','track_name','state','dist','runtime','place','margin','bfSP','prev_race_track']]

In [ ]:
track_stats = gru_in[['track_name','prev_race_track','state', 'prev_race_state']]
track_stats.to_csv('track_stats_for_r.csv')

In [ ]:
r_stats.to_csv('all_stats_for_r.csv')

In [ ]:
gru_ric = gru_in[gru_in['track_name']=="Richmond Straight"]
ris_ohe = gru_ric.trackOHE.iloc[1]

In [ ]:
all_stats = pd.read_excel(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\model_all_price\zany-wood-268 - all_price_df.xlsx")

In [ ]:
all_stats

,Unnamed: 0,flat_dogs,flat_races,prices,pred_price,onehot_win,split_margin,flat_track,flat_date,flat_grade,...,frac kel,frac profit,profit,flat_profit,win_price,colour,profit < 30,outlay < 30,fk profit < 30,fk outlay < 30
0,0,596752133,810746280,38.000000,7.249331,0,8.25,Bulli,2022-08-02,Maiden,...,11.525807,-11.525807,-11.162797,0.0,0.000000,loss,0.000000,0.000000,0.000000,0.000000
1,1,540582618,810746280,23.000000,6.489338,0,20.25,Bulli,2022-08-02,Maiden,...,11.732062,-11.732062,-11.062066,0.0,0.000000,loss,-11.062066,11.062066,-11.732062,11.732062
2,2,491176052,810746280,29.000000,8.899121,0,9.25,Bulli,2022-08-02,Maiden,...,8.176274,-8.176274,-7.788789,0.0,0.000000,loss,-7.788789,7.788789,-8.176274,8.176274
3,3,448840890,810746280,83.519159,7.053618,0,14.00,Bulli,2022-08-02,Maiden,...,13.149539,-13.149539,-12.979792,0.0,0.000000,loss,0.000000,0.000000,0.000000,0.000000
4,5,638803691,810746280,1.779061,7.886688,1,0.00,Bulli,2022-08-02,Maiden,...,0.000000,0.000000,0.000000,0.0,1.779061,no bet - win,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46419,51787,675307102,870731127,3.220901,5.520689,0,0.75,The Gardens,2023-02-11,Grade 5,...,0.000000,0.000000,0.000000,0.0,0.000000,no bet,0.000000,0.000000,0.000000,0.000000
46420,51788,555069004,870731127,8.154917,7.856202,0,7.25,The Gardens,2023-02-11,Grade 5,...,2.027130,-2.027130,-0.466256,0.0,0.000000,loss,-0.466256,0.466256,-2.027130,2.027130
46421,51789,707215615,870731127,3.650000,6.343011,0,7.75,The Gardens,2023-02-11,Grade 5,...,0.000000,0.000000,0.000000,0.0,0.000000,no bet,0.000000,0.000000,0.000000,0.000000
46422,51790,442567379,870731127,17.960270,8.123762,0,10.00,The Gardens,2023-02-11,Grade 5,...,7.427100,-7.427100,-6.741723,0.0,0.000000,loss,-6.741723,6.741723,-7.427100,7.427100


In [ ]:
all_stats['delta'] = all_stats['pred_prob']-all_stats['imp_prob']

In [ ]:
all_stats['lay'] = all_stats['delta'].apply(lambda x: 1 if x <0 else 0)
all_stats['lay amount'] = all_stats.apply(lambda x: x['delta']*-100 if (x['delta']<0 and x['prices']<30) else 0, axis =1)
all_stats['lay liability'] = all_stats.apply(lambda x: x['lay amount']*(x['prices']-1), axis =1)

all_stats['lay profit'] = all_stats.apply(lambda x: x['lay amount'] if x['onehot_win']==0 else -x['lay liability'], axis =1)

In [ ]:
all_stats_day_track = all_stats.groupby(['flat_track','flat_date'], as_index=False).sum()

C:\Users\Nick\AppData\Local\Temp\ipykernel_25148\394905707.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  all_stats_day_track = all_stats.groupby(['flat_track','flat_date'], as_index=False).sum()


In [ ]:
all_stats_day_track = all_stats.groupby(['flat_date'],as_index=False).sum()
all_stats_day_track['roi'] = all_stats_day_track.apply(lambda x: x['profit < 30']/x['outlay < 30'], axis =1)
all_stats_day_track['pos roi'] = all_stats_day_track.apply(lambda x: 1 if x['roi']>0 else 0, axis =1)

C:\Users\Nick\AppData\Local\Temp\ipykernel_25148\1321885639.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  all_stats_day_track = all_stats.groupby(['flat_date'],as_index=False).sum()


In [ ]:
all_stats_day_track['pos roi'].value_counts()

1    96
0    92
Name: pos roi, dtype: int64

In [ ]:
all_stats = pd.read_excel(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\model_all_price\vibrant-energy-237 - all_price_df.xlsx")

all_stats_day_track = all_stats.groupby(['flat_date'],as_index=False).sum()
all_stats_day_track['roi'] = all_stats_day_track.apply(lambda x: x['profit < 30']/x['outlay < 30'], axis =1)
all_stats_day_track['pos roi'] = all_stats_day_track.apply(lambda x: 1 if x['roi']>0 else 0, axis =1)
all_stats_day_track['pos roi'].value_counts()

C:\Users\Nick\AppData\Local\Temp\ipykernel_25148\755784840.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  all_stats_day_track = all_stats.groupby(['flat_date'],as_index=False).sum()


0    94
1    94
Name: pos roi, dtype: int64

In [ ]:
gru_ric = gru_in[gru_in['track_name']=="Richmond"]
ric_ohe = gru_ric.trackOHE.iloc[1]

In [ ]:
lens = gru_in.trackOHE.apply(lambda x: len(x)).value_counts()
lens

69    1154167
Name: trackOHE, dtype: int64

In [ ]:
prev_full = pd.read_pickle(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\results-df-merged-prices.npy")

In [ ]:
prev_full.columns

Index(['@id_x', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice',
       'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments',
       'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId',
       'TrainerName', 'DogId', '@id_y', 'RaceNum', 'RaceName', 'RaceTime',
       'Distance', 'RaceGrade', 'Track_ft', 'State', 'track_code', 'date',
       'DogGrade', 'check', 'SplitTimes', 'minMargin', 'tracksOnehot',
       'dist_x', 'run_time', 'split_margins', 'place', 'track_id',
       'Unnamed: 0', 'dist_y', 'mean_time', 'mean_split', 'split_dist_estim',
       'split_dist', 'dateF1', 'speed', 'split_speed', 'box', 'margin',
       'weigth', 'dateF', 'dog_name', 'EVENT_ID', 'EVENT_DT', 'SELECTION_ID',
       'BSP', 'dog', 'MENU_HINT', 'weight', 'dist_round', 'Track',
       'StartPrice_num', 'PIR_adj', 'start_pos', 'last_pos', 'first_out',
       'pos_change', 'race_time'],
      dtype='object')

In [ ]:
trakcs = prev_full['Track'].value_counts()

In [ ]:
prev_full = prev_full[prev_full['State']=="NSW"]

In [ ]:
x = prev_full['StartPrice'].value_counts()
x = prev_full['StartPrice_num'].value_counts()

: 

In [ ]:
y =prev_full[prev_full['StartPrice_num']==0]

In [ ]:
pd.to_numeric(prev_full['StartPrice'].apply(lambda x:x.replace('$','').replace('F','') if isinstance(x,str) else x), errors='coerce')

0         4.7
1         2.8
2         3.7
3        32.0
4        15.8
         ... 
31679     5.0
31688    19.0
31697     6.0
31706    51.0
31715    61.0
Name: StartPrice, Length: 1185270, dtype: float64

In [ ]:
prev_full['PIR_adj'] = prev_full.PIR.str.replace('[^0-9]', '').fillna(0)
prev_full['start_pos'] =  prev_full['PIR_adj'].apply(lambda x: int(x[0]) if x else 8)
prev_full['last_pos'] =  prev_full['PIR_adj'].apply(lambda x: int(x[-1]) if x else 8)
prev_full['first_out'] = prev_full['start_pos'].apply(lambda x: 1 if x==1 else 0)
prev_full['pos_change'] = pd.to_numeric(prev_full['start_pos']) - pd.to_numeric(prev_full['last_pos'])

C:\Users\Nick\AppData\Local\Temp\ipykernel_8988\3985882909.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  prev_full['PIR_adj'] = prev_full.PIR.str.replace('[^0-9]', '').fillna(0)


In [ ]:
pir_testing = prev_full[prev_full['State']=="NSW"][['PIR','PIR_adj','start_pos','last_pos','first_out','pos_change']]

In [ ]:
prev_full_track = prev_full.groupby(['State','Track'])

In [ ]:
check_tracks = prev_full_track.PIR_adj.value_counts()

In [ ]:
tracks_dist = prev_full.groupby(['track_id']).mean()

C:\Users\Nick\AppData\Local\Temp\ipykernel_8988\3535940953.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tracks_dist = prev_full.groupby(['track_id']).mean()


In [ ]:
prev_full = prev_full.merge(tracks, on='track_code')

In [ ]:
full_times_for_r = prev_full[['Track','dist_x','SplitTimes','RunTime']]

In [ ]:
full_times_for_r.to_csv('full_times_for_r.csv')

In [ ]:
prev_full['SplitTimes'].value_counts()

6.76     5207
6.75     5132
6.77     5111
6.78     5052
6.79     5006
         ... 
28.01       1
28.61       1
19.15       1
21.86       1
20.99       1
Name: SplitTimes, Length: 1984, dtype: int64

In [ ]:
prev_full = prev_full.drop(columns = ['state','Track'])

In [ ]:
prev_full = prev_full.rename(columns={'track_name':'Track'})

In [ ]:
prev_full.columns

Index(['@id_x', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice',
       'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments',
       'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId',
       'TrainerName', 'DogId', '@id_y', 'RaceNum', 'RaceName', 'RaceTime',
       'Distance', 'RaceGrade', 'Track_ft', 'State', 'track_code', 'date',
       'DogGrade', 'check', 'SplitTimes', 'minMargin', 'tracksOnehot',
       'dist_x', 'run_time', 'split_margins', 'place', 'track_id',
       'Unnamed: 0', 'dist_y', 'mean_time', 'mean_split', 'split_dist_estim',
       'split_dist', 'dateF1', 'speed', 'split_speed', 'box', 'margin',
       'weigth', 'dateF', 'dog_name', 'EVENT_ID', 'EVENT_DT', 'SELECTION_ID',
       'BSP', 'dog', 'MENU_HINT', 'weight', 'dist_round', 'Track'],
      dtype='object')

In [ ]:
prev_full[prev_full['Track']=="Richmond Straight"]

,@id_x,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,...,dog_name,EVENT_ID,EVENT_DT,SELECTION_ID,BSP,dog,MENU_HINT,weight,dist_round,Track
1130965,451767897,1.0,HIATUS,4,4,35.1,2.2,None,2.25,None,...,HIATUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Richmond Straight
1130966,504048122,2.0,TWO CAN TOMMY,8,8,33.7,6.0,None,2.25,2.29,...,TWO CAN TOMMY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Richmond Straight
1130967,465423148,3.0,MELLENCAMP,2,9,30.3,34.0,None,9.00,6.71,...,MELLENCAMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Richmond Straight
1130968,439998744,4.0,BAD BOY SLASH,3,3,35.6,5.5,None,9.25,0.43,...,BAD BOY SLASH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Richmond Straight
1130969,465423126,5.0,KNIGHT RIDER,5,5,34.6,6.5,None,9.25,0.00,...,KNIGHT RIDER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Richmond Straight
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138529,NaN,4.0,SEE HOW,4,4,35.7,18.0,None,3.5,0.00,...,SEE HOW,NaN,NaN,NaN,NaN,NaN,NaN,35.7,300.0,Richmond Straight
1138530,NaN,5.0,LUCKY ROMY,3,9,29.3,101.0,None,4.0,0.57,...,LUCKY ROMY,NaN,NaN,NaN,NaN,NaN,NaN,29.3,300.0,Richmond Straight
1138531,NaN,6.0,FULL PELT,6,6,32.0,34.0,None,4.25,0.43,...,FULL PELT,NaN,NaN,NaN,NaN,NaN,NaN,32.0,300.0,Richmond Straight
1138532,NaN,7.0,ORANMORE,5,5,28.6,31.0,None,5.75,1.43,...,ORANMORE,NaN,NaN,NaN,NaN,NaN,NaN,28.6,300.0,Richmond Straight


In [ ]:
prev_full.to_pickle(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\results-df-merged-prices.npy")

NameError: name 'prev_full' is not defined

In [ ]:
greys = ft.Fasttrack(SECKEY)

Valid Security Key


In [ ]:
tracks = greys.listTracks()
tracks

,track_name,track_code,state
0,Albury,223,NSW
1,Armidale,225,NSW
2,Bathurst,226,NSW
3,Broke Hill,227,NSW
4,Bulli,202,NSW
...,...,...,...
110,Warrnambool,313,VIC
111,Warrnambool,363,VIC
112,Cannington,600,WA
113,Mandurah,601,WA


In [ ]:
gru_in.track_code.value_counts()

900    52255
400    50980
600    48435
601    47981
407    44486
214    38721
312    36356
306    35281
302    34682
308    33152
303    31384
249    30613
305    30101
500    27967
502    25281
313    25186
413    23148
200    21424
309    20287
504    20016
911    18161
304    17960
404    17674
307    16592
901    15444
300    15423
203    15043
233    14571
301    14288
800    13820
412    13763
205    13512
903    13467
206    13279
904    12574
231    12567
226    12490
510    12478
700    12323
201    12316
315    12208
701    11916
314    11264
202    10992
310    10844
235    10608
220    10525
211    10313
208     9950
402     9337
511     8630
702     8499
207     8494
218     7795
251     7569
219     7409
906     7383
905     3067
221     2689
227     2400
229     2202
244     2063
703     1832
242     1749
238     1616
704     1560
237     1237
602     1006
215      945
209      906
248      836
910      118
909       92
228       58
Name: track_code, dtype: int64

In [ ]:
def generate_dog_stats(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.copy()
    original_cols = df_in.columns
    df[f'speed_avg{factor}_{rolling_window}'] = df_g['speed'].rolling(rolling_window, min_periods=1, closed='left').mean()['speed']
    df[f'speed_max{factor}_{rolling_window}'] = df_g['speed'].rolling(rolling_window, min_periods=1, closed='left').max()['speed']
    df[f'split_speed_avg{factor}_{rolling_window}'] = df_g['split_speed'].rolling(rolling_window, min_periods=1, closed='left').mean()['split_speed']
    df[f'split_speed_max{factor}_{rolling_window}'] = df_g['split_speed'].rolling(rolling_window, min_periods=1, closed='left').max()['split_speed']
    df[f'split_margin_avg{factor}_{rolling_window}'] = df_g['split_margins'].rolling(rolling_window, min_periods=1, closed='left').mean()['split_margins']
    df[f'margin_avg{factor}_{rolling_window}'] = df_g['margin'].rolling(rolling_window, min_periods=1, closed='left').mean()['margin']
    df[f'races{factor}_{rolling_window}'] = df_g.cumcount().reset_index(drop=True)
    df[f'wins{factor}_{rolling_window}'] = df_g['win'].rolling(1000, min_periods=1, closed='left').sum()['win']
    return(df)

In [ ]:
def generate_prev_race(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.copy()
    original_cols = df_in.columns
    df[f'prev_race'] = df_g['RaceId'].shift(1).fillna(-1)
    df[f'prev_race_date'] = df_g['dateF1'].shift(1).fillna(-1)
    return(df)

In [ ]:
def update_basic_form_data():
    seckey = SECKEY
    greys = ft.Fasttrack(seckey)
    today = (datetime.today()+timedelta(days=1)).strftime('%Y-%m-%d')
    lastweek = (datetime.today()-timedelta(days=14)).strftime('%Y-%m-%d')

    race_details, dog_results = greys.getBasicFormat(lastweek, dt_end=today)
    race_details['RaceId'] = race_details['@id']
    dog_results['DogId'] = dog_results['@id']
    dog_results['box'] = dog_results['RaceBox']
    basic_form = dog_results.merge(race_details,how='left', on='RaceId')[['@id_x', '@id_y', 'DogGrade']]
    basic_form = basic_form.rename(columns={'@id_x':'DogId','@id_y':'RaceId'})
    old_merge = pd.read_pickle(r'./DATA/basic form/all basic form merged.npy')
    all_races = pd.concat([old_merge,basic_form])
    all_races.drop_duplicates(keep='last')
    all_races.to_pickle(r'./DATA/basic form/all basic form merged.npy')
    return all_races

In [ ]:
def round_down(num, divisor):
    return num - (num%divisor)

def generate_prev_race(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.copy()
    original_cols = df_in.columns
    df[f'prev_race'] = df_g['RaceId'].shift(1).fillna(-1)
    df[f'prev_race_date'] = df_g['dateF1'].shift(1).fillna(-1)
    return(df)

In [ ]:
seckey = "50efd775-e988-4be3-924f-87631fabdc3f"
greys = ft.Fasttrack(seckey)
today = datetime.today().strftime('%Y-%m-%d')

race_details, dog_results = greys.getBasicFormat(today, dt_end=today)
race_details['RaceId'] = race_details['@id']
dog_results['DogId'] = dog_results['@id']
dog_results['box'] = dog_results['RaceBox']
basic_form = dog_results.merge(race_details,how='left', on='RaceId')
#basic_form = basic_form.rename(columns={'@id_x':'DogId','@id_y':'RaceId'})

race_details.to_pickle(f'all basic form races{today}.npy')
dog_results.to_pickle(f'all basic form dogs{today}.npy')

prediction_df = pd.merge(dog_results,race_details, how='left', on='RaceId')

# # x = featurecreations.generate_prediction_dataframe("results-df-merged-prices.npy", basic_form, 'encoder')

# x.to_pickle(f'prediction_input {today}.npy')

Valid Security Key
Getting meets for each date ..


100%|██████████| 1/1 [00:00<00:00,  2.02it/s]


Getting dog lineups ..


100%|██████████| 11/11 [00:09<00:00,  1.19it/s]


In [ ]:
all_results = pd.read_pickle('./results-df-merged-prices.npy')

In [ ]:
ohe = OneHotEncoder(sparse=False)

In [ ]:
ohe = OneHotEncoder(sparse=False)
print(len(ohe.fit_transform(all_results[['Track']])[0]))
with open('encoder_new', 'wb') as f:
    pickle.dump(ohe, f)

69


In [ ]:
ohe.categories

'auto'

In [ ]:
len(ohe.fit_transform(all_results[['Track']])[0])

69

In [ ]:
all_results

,@id_x,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,...,dateF,dog_name,EVENT_ID,EVENT_DT,SELECTION_ID,BSP,dog,MENU_HINT,weight,dist_round
0,314367490,1.0,SUPER BIG,6,6,32.0,4.7,None,5.91,None,...,2019-12-01,SUPER BIG,165727592,01-12-2019 00:00,27105996,5.481909,Super Big,AUS / Heal (AUS) 1st Dec,NaN,NaN
1,296096022,2.0,RIO DIXIE,1,1,32.2,2.8,None,5.91,5.91,...,2019-12-01,RIO DIXIE,165727592,01-12-2019 00:00,27105992,3.915809,Rio Dixie,AUS / Heal (AUS) 1st Dec,NaN,NaN
2,304203706,3.0,COCO JUMBO,8,8,29.9,3.7,None,6.99,1.07,...,2019-12-01,COCO JUMBO,165727592,01-12-2019 00:00,27105997,3.740928,Coco Jumbo,AUS / Heal (AUS) 1st Dec,NaN,NaN
3,291188715,4.0,BEARING GOLD,4,4,29.2,32.0,None,8.46,1.47,...,2019-12-01,BEARING GOLD,165727592,01-12-2019 00:00,25017544,55.144763,Bearing Gold,AUS / Heal (AUS) 1st Dec,NaN,NaN
4,317789815,5.0,ROLLIN' IN,3,3,28.2,15.8,None,11.31,2.86,...,2019-12-01,ROLLIN IN,165727592,01-12-2019 00:00,27105994,16.000000,Rollin In,AUS / Heal (AUS) 1st Dec,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115343,NaN,4.0,GATLIN,1,1,31.3,6.0,None,1.5,0.86,...,2023-02-04,GATLIN,NaN,NaN,NaN,NaN,NaN,NaN,31.3,300.0
115347,NaN,5.0,OSCAR MOSES,3,3,37.7,2.7,None,4.75,3.29,...,2023-02-04,OSCAR MOSES,NaN,NaN,NaN,NaN,NaN,NaN,37.7,300.0
115351,NaN,6.0,SACRED STANCE,8,8,35.6,3.2,None,4.75,0.14,...,2023-02-04,SACRED STANCE,NaN,NaN,NaN,NaN,NaN,NaN,35.6,300.0
115355,NaN,7.0,DOONSIDE ANDY,2,2,34.6,81.0,None,6.0,1.29,...,2023-02-04,DOONSIDE ANDY,NaN,NaN,NaN,NaN,NaN,NaN,34.6,300.0


In [ ]:
ohe = pickle.load(open('encoder', 'rb'))

In [ ]:
ohe = pickle.load(open('encoder', 'rb'))

pred_track_OHE = ohe.transform(prediction_df[["Track"]])

prediction_df["tracksOnehot"] = pred_track_OHE.tolist()
prediction_df["dist"] = prediction_df["Distance"].astype(str).str[:-1].astype(float)
prediction_df['dist_round'] = prediction_df.dist.apply(lambda x: round_down(x, 50))
prediction_df["date"] =  pd.to_datetime(prediction_df["Date"], format="%d %b %y").dt.date
prediction_df['DogGrade'] = pd.to_numeric(prediction_df['DogGrade'],errors='coerce').fillna(8)
prediction_df['Box'] = pd.to_numeric(prediction_df['box'],errors='coerce').fillna(8)
race_pred_forms = prediction_df.sort_values("date", ascending=False).groupby(["RaceId"], sort=False)
form = pd.concat([all_results, prediction_df])


In [ ]:
seckey = "50efd775-e988-4be3-924f-87631fabdc3f"
greys = ft.Fasttrack(seckey)
today = datetime.today().strftime('%Y-%m-%d')

race_details, dog_results = greys.getBasicFormat(today, dt_end=today)
race_details['RaceId'] = race_details['@id']
dog_results['DogId'] = dog_results['@id']
dog_results['box'] = dog_results['RaceBox']
basic_form = dog_results.merge(race_details,how='left', on='RaceId')
#basic_form = basic_form.rename(columns={'@id_x':'DogId','@id_y':'RaceId'})

race_details.to_pickle(f'all basic form races{today}.npy')
dog_results.to_pickle(f'all basic form dogs{today}.npy')

prediction_df = pd.merge(dog_results,race_details, how='left', on='RaceId')

Valid Security Key
Getting meets for each date ..


100%|██████████| 1/1 [00:00<00:00,  2.07it/s]


Getting dog lineups ..


100%|██████████| 11/11 [00:08<00:00,  1.36it/s]


In [ ]:
prediction_df

,@id_x,RaceBox,Last5,DogName,BestTime,DogHandicap,Odds,Rating,Speed,DogComment,...,VICGREYS,RaceComment,Track_ft,Track,State,track_code,Date,Quali,TipsComments_Bet,TipsComments_Tips
0,710477883,1,None,REASONS UNKNOWN,FSH,None,$4.80,0,None,"Well bred, drawn to get the breaks, danger",...,None,"""IDYLLIC EAGLE (2) has been smartly placed in ...",Bendigo,Bendigo,VIC,303,04 Feb 23,None,"Quinella 2/1,3,5 ($10 for 333.33%)","2, 1, 5, 3"
1,640983413,2,22,IDYLLIC EAGLE,FSH,None,$2.40,0,65.57,"Resuming, shown some talent, expected to fire",...,None,"""IDYLLIC EAGLE (2) has been smartly placed in ...",Bendigo,Bendigo,VIC,303,04 Feb 23,None,"Quinella 2/1,3,5 ($10 for 333.33%)","2, 1, 5, 3"
2,717251649,3,None,SO RECKLESS,FSH,None,$6.00,0,None,"Dam has produced some capable types, respect",...,None,"""IDYLLIC EAGLE (2) has been smartly placed in ...",Bendigo,Bendigo,VIC,303,04 Feb 23,None,"Quinella 2/1,3,5 ($10 for 333.33%)","2, 1, 5, 3"
3,712482121,4,None,REALITY TEARS,FSH,None,$10.00,0,None,"Well bred but the draw is a concern, watch bet...",...,None,"""IDYLLIC EAGLE (2) has been smartly placed in ...",Bendigo,Bendigo,VIC,303,04 Feb 23,None,"Quinella 2/1,3,5 ($10 for 333.33%)","2, 1, 5, 3"
4,701922546,5,2744,REDEEMER RUSLIE,FSH,None,$8.00,92,62.1,"No luck since his debut placing, can atone, th...",...,None,"""IDYLLIC EAGLE (2) has been smartly placed in ...",Bendigo,Bendigo,VIC,303,04 Feb 23,None,"Quinella 2/1,3,5 ($10 for 333.33%)","2, 1, 5, 3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014,634243961,4,44223,BATTLEFIELD,25.20,None,None,92,61.848,None,...,None,"""""",Ipswich,Ipswich,QLD,407,04 Feb 23,None,None,None
1015,576142577,5,45655,ALPHA HUMBUG,NBT,None,None,87,63.009,None,...,None,"""""",Ipswich,Ipswich,QLD,407,04 Feb 23,None,None,None
1016,694775795,6,24676,SIDE KICK,25.75,None,None,100,65.247,None,...,None,"""""",Ipswich,Ipswich,QLD,407,04 Feb 23,None,None,None
1017,578900016,7,63885,GRACE'S SECRET,25.28,None,None,87,64.592,None,...,None,"""""",Ipswich,Ipswich,QLD,407,04 Feb 23,None,None,None


In [ ]:
def update_results_data():
    seckey = SECKEY
    greys = ft.Fasttrack(seckey)
    today = (datetime.today()+timedelta(days=1)).strftime('%Y-%m-%d')
    lastweek = (datetime.today()-timedelta(days=14)).strftime('%Y-%m-%d')
    race_details, dog_results = greys.getRaceResults(lastweek, dt_end=today)
    print(f"race detail cols = {race_details.columns}, dog detail cols = {dog_results.columns}")
    race_details = race_details.rename(columns={'@id':'RaceId'})
    dog_results = dog_results.rename(columns={'@id':'DogId'})
    all_results = dog_results.merge(race_details, how='left', on='RaceId')
    all_results = all_results[all_results['Place']!=None]
    return all_results

In [ ]:
def update_basic_form_data():
    seckey = SECKEY
    greys = ft.Fasttrack(seckey)
    today = (datetime.today()+timedelta(days=1)).strftime('%Y-%m-%d')
    lastweek = (datetime.today()-timedelta(days=14)).strftime('%Y-%m-%d')

    race_details, dog_results = greys.getBasicFormat(lastweek, dt_end=today)
    race_details['RaceId'] = race_details['@id']
    dog_results['DogId'] = dog_results['@id']
    dog_results['box'] = dog_results['RaceBox']
    basic_form = dog_results.merge(race_details,how='left', on='RaceId')[['@id_x', '@id_y', 'DogGrade']]
    basic_form = basic_form.rename(columns={'@id_x':'DogId','@id_y':'RaceId'})
    old_merge = pd.read_pickle(r'./DATA/basic form/all basic form merged.npy')
    all_races = pd.concat([old_merge,basic_form])
    all_races.drop_duplicates(keep='last')
    all_races.to_pickle(r'./DATA/basic form/all basic form merged.npy')
    return all_races

In [ ]:
betfair_sp_file = (
    r"./DATA/df-betfairSP.npy"
)
previous_results_file = (
    r"./DATA/all-results-db.npy"
)
split_dist_file = (
    r"./DATA/split_dist_ft.csv"
)
prev_full_details = r"results-df-merged-prices.npy"

prev_gru_df = pd.read_pickle(r"./DATA/gru_input.npy")
df = pd.read_pickle(betfair_sp_file)



last_date = prev_gru_df.date.max()

print(f"Database dated until {last_date}")

start_date = last_date

dog_results = update_results_data()
basic_form = update_basic_form_data()

dog_results = dog_results.merge(basic_form, how='left', on=['RaceId', 'DogId'])



Database dated until 2023-02-03 00:00:00
Valid Security Key
Getting meets for each date ..


100%|██████████| 16/16 [00:07<00:00,  2.00it/s]


Getting historic results details ..


100%|██████████| 207/207 [02:12<00:00,  1.56it/s]


race detail cols = Index(['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade',
       'Track_ft', 'Track', 'State', 'track_code', 'date'],
      dtype='object'), dog detail cols = Index(['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice',
       'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments',
       'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId',
       'TrainerName'],
      dtype='object')
Valid Security Key
Getting meets for each date ..


...

 19%|█▉        | 3/16 [00:37<03:37, 16.73s/it]

skipping date 23_01_2023


100%|██████████| 16/16 [00:43<00:00,  2.71s/it]


Getting dog lineups ..


100%|██████████| 194/194 [02:23<00:00,  1.35it/s]


In [ ]:
importlib.reload(featurecreations)
x = featurecreations.generate_results_df_v2(prev_full_details, dog_results, 'na', betfair_sp_file, split_dist_file,mode='', prev_df=prev_gru_df, ohe='encoder')

print(x)
x.to_pickle(r"./DATA/new_windows_gru_REAL_real.npy")

173856
Index(['DogId', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice',
       'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments',
       'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId',
       'TrainerName', 'RaceNum', 'RaceName', 'RaceTime', 'Distance',
       'RaceGrade', 'Track_ft', 'Track', 'State', 'track_code', 'date',
       'DogGrade', 'check', 'SplitTimes', 'minMargin', 'tracksOnehot',
       'dist_x', 'run_time', 'split_margins', 'place', 'track_id',
       'Unnamed: 0', 'dist_y', 'mean_time', 'mean_split', 'split_dist_estim',
       'split_dist'],
      dtype='object')
starting stats rolling window
             dogid        dog_name     raceid race_grade        date  \
0        314367490       SUPER BIG  501007024  Mixed 6/7  2019-12-01   
1        141319750  SPRING SPINNER  500541116  Mixed 4/5  2019-12-01   
2        240988036   MY GIRL COURT  500541116  Mixed 4/5  2019-12-01   
3        312552897   BOTANY THOMMO  500543140    Class 1  20

In [ ]:
dog_results['Place'] = dog_results['Place'].apply(lambda x: x[0] if isinstance(x, str) else x)
dog_results['Place'] = pd.to_numeric(dog_results['Place'], errors='coerce')

In [ ]:
clock = x[x['dog_name']=='CLOCK OFF']

In [ ]:
dog_results['check'] = dog_results['Place'].apply(lambda x: 1 if isinstance(x, str) else 0)
dog_results = dog_results[dog_results['check']==1] 

In [ ]:
dog_results

NameError: name 'dog_results' is not defined

In [ ]:
prev_full_details.Place

AttributeError: 'str' object has no attribute 'Place'

In [ ]:
x = pd.read_pickle('./DATA/new_windows_gru_REAL.npy')
fuel = x[x['dogid']=='638803194']

In [ ]:
all_results = pd.read_pickle(r"results-df-merged-prices.npy")
all_results['win'] = all_results['Place'].apply(lambda x: 1 if x==1 else 0)
all_results['count'] = 1

In [ ]:
all_results = all_results.reset_index(drop=True)

In [ ]:
dog_grouped = all_results.groupby(['DogId'], sort=False,as_index=False)

In [ ]:
test = generate_dog_stats(all_results,dog_grouped)

In [ ]:
fuel_stats = test[test['DogId']=='638803194']